In [4]:
import os
import flask
import requests
import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery

CLIENT_SECRETS_FILE = "client_secret_144362790558-03dalg61mpjbr28psk0ivg68qmd2mmop.apps.googleusercontent.com.json"

SCOPES = ['https://www.googleapis.com/auth/fitness.activity.read', 
          'https://www.googleapis.com/auth/fitness.activity.write', 
          'https://www.googleapis.com/auth/fitness.blood_glucose.read',
          'https://www.googleapis.com/auth/fitness.blood_pressure.read',
          'https://www.googleapis.com/auth/fitness.body.read',
          'https://www.googleapis.com/auth/fitness.body_temperature.read',
          'https://www.googleapis.com/auth/fitness.location.read',
          'https://www.googleapis.com/auth/fitness.nutrition.read',
          'https://www.googleapis.com/auth/fitness.oxygen_saturation.read',
          'https://www.googleapis.com/auth/fitness.reproductive_health.read']


API_SERVICE_NAME = 'fitness'
API_VERSION = 'v1'

app = flask.Flask(__name__)
app.secret_key = 'hkg1234#'

DATA_SOURCE = "derived:com.google.active_minutes:com.google.android.gms:from_activity<-merge_activity_segments"
API_USER_ID="me"

#DATA_SET = "1570660988126-1590660988126"

def today_dataset_start():
    today = datetime.today().date()
    return int(time.mktime(today.timetuple()) * 1000000000)

def today_dataset_end():
    now = datetime.today()
    return int(time.mktime(now.timetuple()) * 1000000000) 

def get_datasources(self, data_type_name):
        datasources_request = self.users().dataSources().list(
            userId=API_USER_ID,
            dataTypeName=data_type_name,
        )
        data = datasources_request.execute()
        return data.get('dataSource')
    
def _get_dataset(self, source):
        dataset = "%s-%s" % (_today_dataset_start(), _today_dataset_end())

        return self.users().dataSources(). \
            datasets(). \
            get(userId=API_USER_ID, dataSourceId=source, datasetId=dataset). \
            execute()

@app.route('/')
def index():
  return print_index_table()


@app.route('/test')
def test_api_request():
  if 'credentials' not in flask.session:
    return flask.redirect('authorize')

  credentials = google.oauth2.credentials.Credentials(
      **flask.session['credentials'])

  fitness = googleapiclient.discovery.build(
      API_SERVICE_NAME, API_VERSION, credentials=credentials)

  weight_datasources = get_datasources(fitness,'com.google.weight');
    
    
  weight_datapoints = {}
  for datasource in weight_datasources:
            datasource_id = datasource.get('dataStreamId')
            weight_request = fitness.users().dataSources().\
                dataPointChanges().list(
                    userId=API_USER_ID,
                    dataSourceId=datasource_id,
                )
            
            weight_data = weight_request.execute()
            weight_inserted_datapoints = weight_data.get('insertedDataPoint')

            for datapoint in weight_inserted_datapoints:
                point_value = datapoint.get('value')
                if not point_value:
                    continue
                weight = point_value[0].get('fpVal')
                if not weight:
                    continue
                weight = round(weight, 2)
                print(weight)
                last_update_milis = int(datapoint.get('modifiedTimeMillis', 0))
                if not last_update_milis:
                    continue
                weight_datapoints[last_update_milis] = weight
            
  print("Gathered")
  for data in weight_datapoints:
    print(data)

  files = fitness.users().dataSources().list(userId=API_USER_ID).execute()
 
  flask.session['credentials'] = credentials_to_dict(credentials)

  return flask.jsonify(**files)


@app.route('/authorize')
def authorize():
  
  flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
      CLIENT_SECRETS_FILE, scopes=SCOPES)

  flow.redirect_uri = flask.url_for('oauth2callback', _external=True)

  authorization_url, state = flow.authorization_url(
      
      include_granted_scopes='false')

  
  flask.session['state'] = state

  return flask.redirect(authorization_url)


@app.route('/oauth2callback')
def oauth2callback():
  
  state = flask.session['state']

  flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
      CLIENT_SECRETS_FILE, scopes=SCOPES, state=state)
  flow.redirect_uri = flask.url_for('oauth2callback', _external=True)

  authorization_response = flask.request.url
  flow.fetch_token(authorization_response=authorization_response)

  credentials = flow.credentials
  flask.session['credentials'] = credentials_to_dict(credentials)

  return flask.redirect(flask.url_for('test_api_request'))


@app.route('/revoke')
def revoke():
  if 'credentials' not in flask.session:
    return ('You need to <a href="/authorize">authorize</a> before ' +
            'testing the code to revoke credentials.')

  credentials = google.oauth2.credentials.Credentials(
    **flask.session['credentials'])

  revoke = requests.post('https://oauth2.googleapis.com/revoke',
      params={'token': credentials.token},
      headers = {'content-type': 'application/x-www-form-urlencoded'})

  status_code = getattr(revoke, 'status_code')
  if status_code == 200:
    return('Credentials successfully revoked.' + print_index_table())
  else:
    return('An error occurred.' + print_index_table())


@app.route('/clear')
def clear_credentials():
  if 'credentials' in flask.session:
    del flask.session['credentials']
  return ('Credentials have been cleared.<br><br>' +
          print_index_table())


def credentials_to_dict(credentials):
  return {'token': credentials.token,
          'refresh_token': credentials.refresh_token,
          'token_uri': credentials.token_uri,
          'client_id': credentials.client_id,
          'client_secret': credentials.client_secret,
          'scopes': credentials.scopes}

def print_index_table():
  return ('<table>' +
          '<tr><td><a href="/test">Test an API request</a></td>' +
          '<td>Submit an API request and see a formatted JSON response. ' +
          '    Go through the authorization flow if there are no stored ' +
          '    credentials for the user.</td></tr>' +
          '<tr><td><a href="/authorize">Test the auth flow directly</a></td>' +
          '<td>Go directly to the authorization flow. If there are stored ' +
          '    credentials, you still might not be prompted to reauthorize ' +
          '    the application.</td></tr>' +
          '<tr><td><a href="/revoke">Revoke current credentials</a></td>' +
          '<td>Revoke the access token associated with the current user ' +
          '    session. After revoking credentials, if you go to the test ' +
          '    page, you should see an <code>invalid_grant</code> error.' +
          '</td></tr>' +
          '<tr><td><a href="/clear">Clear Flask session credentials</a></td>' +
          '<td>Clear the access token currently stored in the user session. ' +
          '    After clearing the token, if you <a href="/test">test the ' +
          '    API request</a> again, you should go back to the auth flow.' +
          '</td></tr></table>')


if __name__ == '__main__':
  
  os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

  
  app.run('localhost', 8000, debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://localhost:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/May/2020 15:34:20] "GET / HTTP/1.1" 200 -


83
84
84
83
Gathered
1590232827511
1590186030423
1590140595658
1590232824560


127.0.0.1 - - [30/May/2020 15:34:24] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2020 16:04:27] "GET / HTTP/1.1" 200 -


83
84
84
83
Gathered
1590232827511
1590186030423
1590140595658
1590232824560


127.0.0.1 - - [30/May/2020 16:04:31] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2020 17:03:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/May/2020 10:32:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/May/2020 12:35:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/May/2020 12:47:58] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
files = fitness.users().dataSources().datasets().get(userId='me',dataSourceId=DATA_SOURCE,datasetId=DATA_SET).execute()

In [ ]:
POST https://www.googleapis.com/fitness/v1/users/[USERID]/dataset:aggregate?key=[YOUR_API_KEY] HTTP/1.1

Authorization: Bearer [YOUR_ACCESS_TOKEN]
Accept: application/json
Content-Type: application/json

{
  "aggregateBy": [
    {}
  ],
  "endTimeMillis": 0,
  "startTimeMillis": 0
}


In [ ]:
pip install oauth2client

In [ ]:
import json
import httplib2

from datetime import datetime
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow

CLIENT_ID = '144362790558-03dalg61mpjbr28psk0ivg68qmd2mmop.apps.googleusercontent.com'
CLIENT_SECRET = 'LWMUzqFSGih6bOd_dWufxa5M'

OAUTH_SCOPE = 'https://www.googleapis.com/auth/fitness.activity.read'

DATA_SOURCE = "derived:com.google.step_count.delta:com.google.android.gms:aggregated"


DATA_SET = "1590088022000-1590174422000"

REDIRECT_URI = 'http://localhost:8000'

def retrieve_data():
    """
    Run through the OAuth flow and retrieve credentials.
    Returns a dataset (Users.dataSources.datasets):
    https://developers.google.com/fit/rest/v1/reference/users/dataSources/datasets
    """
    flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
    authorize_url = flow.step1_get_authorize_url()
    print ('Go to the following link in your browser:')
    print (authorize_url)
    code = input('Enter verification code: ').strip()
    credentials = flow.step2_exchange(code)

    http = httplib2.Http()
    http = credentials.authorize(http)

    fitness_service = build('fitness', 'v1', http=http)

    return fitness_service.users().dataSources(). \
              datasets(). \
              get(userId='me', dataSourceId=DATA_SOURCE, datasetId=DATA_SET). \
              execute()

def nanoseconds(nanotime):
    """
    Convert epoch time with nanoseconds to human-readable.
    """
    dt = datetime.fromtimestamp(nanotime // 1000000000)
    return dt.strftime('%Y-%m-%d %H:%M:%S')

if __name__ == "__main__":
    dataset = retrieve_data()
    with open('dataset.txt', 'w') as outfile:
        json.dump(dataset, outfile)

    last_point = dataset["point"][-1]
    print ("Start time:", nanoseconds(int(last_point.get("startTimeNanos", 0))))
    print ("End time:", nanoseconds(int(last_point.get("endTimeNanos", 0))))
    print ("Data type:", last_point.get("dataTypeName", None))
    print ("Steps:", last_point["value"][0].get("intVal", None))

In [ ]:
def _get_dataset(self, source):
        dataset = "%s-%s" % (_today_dataset_start(), _today_dataset_end())

        return self.users().dataSources(). \
            datasets(). \
            get(userId=API_USER_ID, dataSourceId=source, datasetId=dataset). \
            execute()
    

            DATA_SOURCE = "derived:com.google.step_count.delta:" \
                  "com.google.android.gms:estimated_steps"
        values = []
        for point in fitness._get_dataset(fitness.DATA_SOURCE)["point"]:
            if int(point["startTimeNanos"]) > _today_dataset_start():
                values.append(point['value'][0]['intVal'])

        print(sum(values))
        